In [1]:
!pip install geopandas
!pip install osmnx
!pip install scipy
!pip installn networkx

ERROR: unknown command "installn" - maybe you meant "install"



In [12]:
import pickle

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

## Istruzione

Utilizzeremo i seguenti dati:
1. **Biblioteche**
2. **Scuole**
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

In [13]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_BIBLIOTECHE_CLEAN),
     "tipo": "Biblioteche",
     "gdf":  gpd.read_file(PATH_SCUOLE_CLEAN),

     
     "tipo": "Scuole"}
     ]
Graph_functions.auto_analysis_poi(gdf_list, PATH_GEOJSON="../Data/Clean/Analisi/Istruzione.geojson")

Analisi lunghezza strade create


In [14]:
gdf_analisi_istruzione = gpd.read_file("../Data/Clean/Analisi/Istruzione.geojson", driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [16]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_istruzione_mod = gdf_analisi_istruzione[gdf_analisi_istruzione["artificial"] != True]

# Leggi i due POI
gdf_scuole = gpd.read_file(PATH_SCUOLE_CLEAN)
gdf_biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN)

# Unisci in un'unica geometria
geometry_poi = gpd.GeoSeries(
    gpd.pd.concat([gdf_scuole.geometry, gdf_biblioteche.geometry], ignore_index=True)
).union_all()

# Filtra il GDF di partenza
gdf_senza_poi = []
for idx, row in gdf_analisi_istruzione_mod.iterrows():
    if not geometry_poi.contains(row.geometry):
        gdf_senza_poi.append(row)

# Crea nuovo GeoDataFrame
gdf_analisi_istruzione_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Salviamo per test di visualizzazione su kepler
gdf_analisi_istruzione_mod.to_file("../Data/Clean/Analisi/istruzione_senza_poi.geojson", driver="GeoJSON")

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_istruzione_mod_most_highways = gdf_analisi_istruzione_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [17]:
# Stampiamo il risultato
print(gdf_analisi_istruzione_mod_most_highways)
print(f"Tot: {gdf_analisi_istruzione_mod_most_highways["length"].sum()}")

                              length
highway                             
footway                   103.275866
cycleway                   83.758404
residential                34.211914
tertiary                   23.518353
secondary                  12.496913
primary                     6.308517
track                       3.156910
service                     1.937384
pedestrian                  1.816089
path                        1.562238
footway steps               0.456524
unclassified                0.297007
secondary_link              0.224449
track service path          0.217139
tertiary_link               0.175544
residential cycleway        0.168887
residential footway         0.168512
footway pedestrian          0.158629
residential pedestrian      0.146171
cycleway path               0.133149
footway service             0.127157
cycleway footway            0.112020
primary_link                0.097783
cycleway service            0.093506
living_street               0.078276
r